## Model prediction
---
### Import Statements 

In [ ]:
import os 
import sys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
from numpy import load
from numpy import savez_compressed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from statistics import mean
from skimage.measure import label
from scipy.ndimage import binary_dilation, binary_closing
import pandas as pd

In [ ]:
import tensorflow as tf

In [ ]:
import import_ipynb
from patches import * 
from losses import *

In [ ]:
import model_lungseg as model_lungseg

In [ ]:
# Select tumor segmentation model (will need to run/predict with both, then ensemble in eval script)
import model_192x160x80 as model_tumorseg
#import model_192x192x96 as model_tumorseg

### Settings

**Tumor Segmentation Model**

In [ ]:
model_name = "" #model name
resample_dims = (1,1,2) # other: (0.9765625*2.08,0.9765625*2.08,1.5*2.08)
patch_size = (192,192,96) # other: (192,160,80)

n_filters = 32
loss_function = bce_dice
instancenorm = False
leakyrelu = True
metrics = [dice_coef, 'accuracy']

step_size = 0.5 # for guassian probability filter (since model generates prediction on patches)
tumorseg_norm = 'z_score' # choose from z_score or min_max

**Lung Segmentation Model**

In [ ]:
lung_model_name = "" #path to lung segmentation model

lung_patch_size = (192,160,80)

lung_loss_function = bce_dice
lung_n_filters = 8
lung_instancenorm = False
lung_leakyrelu = True

lung_metrics = [dice_coef, 'accuracy']
lung_step_size = 0.5 # for guassian probability filter
lungseg_norm = 'z_score' # choose from z_score or minmax

**Dataset for prediction**

In [ ]:
if resample_dims == (0.9765625*2.08,0.9765625*2.08,1.5*2.08):
    CT_dir = "" #path to test_1 or test_2 CTs at above res
    mask_dir = "" #path to test_1 or test_2 masks at above res
elif resample_dims == (1,1,2):
    CT_dir = "" #path to test_1 or test_2 CTs at above res
    mask_dir = "" #path to test_1 or test_2 masks at above res

**Randomization**

In [ ]:
seed = 2020
np.random.seed(seed)
random.seed(seed)  

**Evaluation dataset**

## Functions

In [ ]:
def prepare_for_model(ct_arr, mask, patch_size):
    
    # Check that CT array and mask array shapes are the same
    if (ct_arr.shape != mask.shape):
        raise Exception("CT and mask shapes are different")
    
    # If CT shape is smaller than patch size in any dimension, pad with zeros
    # Not sure if this should be zeros...
    if (ct_arr.shape[0] < patch_size[0]) or (ct_arr.shape[1] < patch_size[1]) or (ct_arr.shape[2] < patch_size[2]):
        padded_ct_arr = np.zeros((max(patch_size[0],ct_arr.shape[0]),
                               max(patch_size[1],ct_arr.shape[1]),
                               max(patch_size[2],ct_arr.shape[2])))

        padded_mask = np.zeros((max(patch_size[0],ct_arr.shape[0]),
                               max(patch_size[1],ct_arr.shape[1]),
                               max(patch_size[2],ct_arr.shape[2])))

        pad_x = max(patch_size[0] - ct_arr.shape[0],0)
        pad_y = max(patch_size[1] - ct_arr.shape[1],0)
        pad_z = max(patch_size[2] - ct_arr.shape[2],0)
        
        padded_ct_arr[pad_x//2:pad_x//2+ct_arr.shape[0],
                    pad_y//2:pad_y//2+ct_arr.shape[1],
                    pad_z//2:pad_z//2+ct_arr.shape[2]] = ct_arr

        padded_mask[pad_x//2:pad_x//2+ct_arr.shape[0],
                    pad_y//2:pad_y//2+ct_arr.shape[1],
                    pad_z//2:pad_z//2+ct_arr.shape[2]] = mask

        ct_arr = padded_ct_arr
        mask = padded_mask
    
    return ct_arr, mask  

In [ ]:
# Define color maps for displaying tumor/prediction images
colors = ["#52BE80", "#F1C40F", "#E74C3C", "#FFFFFF00"]
cmap = ListedColormap(colors)

### Load Model

**NOTE ONLY LOAD TUMOR SEGMENTATION OR LUNG SEGMENTATION MODEL**

In [ ]:
'''IF RUNNING TUMOR SEG MODEL'''
input_vol = model_tumorseg.Input((patch_size+(1,)), name='vol')
model = model_tumorseg.unet(input_vol, n_filters=n_filters, instancenorm=instancenorm, leakyrelu=leakyrelu)
model.compile(optimizer=model_tumorseg.Adam(), loss=loss_function, metrics=metrics)
model.load_weights(model_name)

'''IF RUNNING LUNG SEG MODEL'''
'''Only run lung segmentation model on CTs with (0.9765625*2.08,0.9765625*2.08,1.5*2.08) voxel dimensions - this is what lung seg model was trained on'''
'''
lungseg_input_vol = model_lungseg.Input((lung_patch_size+(1,)), name='vol')
lungseg_model = model_lungseg.unet(lungseg_input_vol, n_filters=lung_n_filters, instancenorm=lung_instancenorm, leakyrelu=lung_leakyrelu)
lungseg_model.compile(optimizer=model_lungseg.Adam(), loss=lung_loss_function, metrics=lung_metrics)
lungseg_model.load_weights(lung_model_name)
'''

### Prediction

**Load test data and set save directory**

In [ ]:
val_ids = next(os.walk(mask_dir))[2]
print("Evaluation set size: ", len(val_ids))

In [ ]:
model_desc_tumorseg = model_name.split("/")[-1].replace(".h5","")
model_desc_lungseg = lung_model_name.split("/")[-1].replace(".h5","")
output_path_tumorseg = "" #path to output directory for predictions
os.mkdir(output_path_tumorseg)

**Predict**

In [ ]:
for n,_id in enumerate(val_ids):
    
    print(_id)
    
    # Load CT and mask, prepare for model
    ct_path = os.path.join(CT_dir, _id)
    mask_path = os.path.join(mask_dir, _id)
    mask = load(mask_path)['arr_0']
    ct_arr = load(ct_path)['arr_0']
    ct_arr = ct_arr.astype('float32') 
    mask = mask.astype('float32') 
    
    orig_mask = np.copy(mask)
    orig_mask_shape = mask.shape
    
    ct_arr, mask = prepare_for_model(ct_arr, mask, patch_size)
    padded_mask_shape = mask.shape
    
    # Z score normalization
    z_score_ct_arr = np.copy(ct_arr)
    z_score_ct_arr -= np.mean(z_score_ct_arr)
    z_score_ct_arr /= np.std(z_score_ct_arr)
    
    # Min/Max normalization
    min_max_ct_arr = np.copy(ct_arr)
    if eval_dataset=='nsclc':
        min_max_ct_arr += 1024
    min_max_ct_arr = np.clip(min_max_ct_arr, 0, 1500)
    min_max_ct_arr /= 1500
    
    # Normalize
    if lungseg_norm == 'min_max':
        lungseg_ct_arr = min_max_ct_arr
    elif lungseg_norm == 'z_score':
        lungseg_ct_arr = z_score_ct_arr
    
    if tumorseg_norm == 'min_max':
        tumorseg_ct_arr = min_max_ct_arr
    elif tumorseg_norm == 'z_score':
        tumorseg_ct_arr = z_score_ct_arr
    
    # Predict
    '''IF RUNNING TUMOR PREDICTION MODEL'''
    prediction = predict_on_patches(ct_vol=tumorseg_ct_arr, model=model, patch_size=patch_size, step_size=step_size)    
        
    unpad = [a_i - b_i for a_i, b_i in zip(padded_mask_shape, orig_mask_shape)]
    
    
    prediction_unpad = prediction[unpad[0]//2:unpad[0]//2+orig_mask_shape[0],
                                  unpad[1]//2:unpad[1]//2+orig_mask_shape[1],
                                  unpad[2]//2:unpad[2]//2+orig_mask_shape[2]]

    savez_compressed(output_path_tumorseg + _id, prediction_unpad)
    
    '''IF RUNNING LUNG SEG MODEL'''
    ''' 
    lungseg_prediction = predict_on_patches(ct_vol=lungseg_ct_arr, model=lungseg_model, patch_size=lung_patch_size, step_size=lung_step_size)
    lungseg_prediction_unpad = lungseg_prediction[unpad[0]//2:unpad[0]//2+orig_mask_shape[0],
                                  unpad[1]//2:unpad[1]//2+orig_mask_shape[1],
                                  unpad[2]//2:unpad[2]//2+orig_mask_shape[2]]
    savez_compressed(output_path_lungseg + _id, lungseg_prediction_unpad)
    '''